# Proyecto A: Optimización en la Planeación de Transporte Vehicular Urbana Para LogistiCo

## Integrantes

- Barrera Toro, Javier Steven
- Rolon Toloza, Julian Santiago

# Implementación del modelo matemático y procesamiento de datos

In [2]:
import pandas as pd
from geopy.distance import geodesic
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Procesamiento de datos

In [3]:
import pandas as pd
from geopy.distance import geodesic
import os

ruta_clientes_csv = 'caso_base/clients.csv'
ruta_depositos_csv = 'caso_base/depots.csv'
df_vehiculos = pd.read_csv('caso_base/vehicles.csv')

directorio_salida_consolidado = 'Proyecto_Caso_Base'
nombre_archivo_ubicaciones_consolidadas = 'locations.csv'
ruta_ubicaciones_consolidadas_csv = os.path.join(directorio_salida_consolidado, nombre_archivo_ubicaciones_consolidadas)

if not os.path.exists(directorio_salida_consolidado):
    os.makedirs(directorio_salida_consolidado)
    print(f"Directorio '{directorio_salida_consolidado}' creado.")

try:
    print(f"Cargando datos de clientes desde: {ruta_clientes_csv}")
    df_clientes = pd.read_csv(ruta_clientes_csv)

    print(f"Cargando datos de depósitos desde: {ruta_depositos_csv}")
    df_depositos = pd.read_csv(ruta_depositos_csv)

except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar el archivo {e.filename}. Asegúrate de que las rutas sean correctas.")
    exit()

df_depositos_ubic = df_depositos[['LocationID', 'Longitude', 'Latitude']].copy()
df_clientes_ubic = df_clientes[['LocationID', 'Longitude', 'Latitude']].copy()

ubicaciones_consolidadas_df = pd.concat([
    df_depositos_ubic,
    df_clientes_ubic
], ignore_index=True)

try:
    ubicaciones_consolidadas_df.to_csv(ruta_ubicaciones_consolidadas_csv, index=False)
    print(f"Archivo de ubicaciones consolidadas guardado en: {ruta_ubicaciones_consolidadas_csv}")
except Exception as e:
    print(f"Error al guardar el archivo de ubicaciones consolidadas: {e}")
    exit()

try:
    print(f"Cargando ubicaciones consolidadas desde: {ruta_ubicaciones_consolidadas_csv}")
    df_ubicaciones_para_matriz = pd.read_csv(ruta_ubicaciones_consolidadas_csv)
except FileNotFoundError:
    print(f"Error: El archivo consolidado '{ruta_ubicaciones_consolidadas_csv}' no fue encontrado.")
    exit()

print("Calculando matriz de distancias...")
lista_ids_ubicaciones = df_ubicaciones_para_matriz['LocationID'].tolist()
n_ubicaciones = len(lista_ids_ubicaciones)
matriz_distancias_lista = [[0.0] * n_ubicaciones for _ in range(n_ubicaciones)]

for i in range(n_ubicaciones):
    fila_origen = df_ubicaciones_para_matriz.iloc[i]
    coord_origen = (fila_origen['Latitude'], fila_origen['Longitude'])
    for j in range(n_ubicaciones):
        if i == j:
            matriz_distancias_lista[i][j] = 0.0
        else:
            fila_dest = df_ubicaciones_para_matriz.iloc[j]
            coord_dest = (fila_dest['Latitude'], fila_dest['Longitude'])
            distancia_km = geodesic(coord_origen, coord_dest).kilometers
            matriz_distancias_lista[i][j] = distancia_km

print("Matriz de distancias calculada.")

df_matriz_distancias = pd.DataFrame(
    matriz_distancias_lista,
    index=lista_ids_ubicaciones,
    columns=lista_ids_ubicaciones
)

print("\nMatriz de Distancias (DataFrame):")
print(df_matriz_distancias)

Cargando datos de clientes desde: caso_base/clients.csv
Cargando datos de depósitos desde: caso_base/depots.csv
Archivo de ubicaciones consolidadas guardado en: Proyecto_Caso_Base\locations.csv
Cargando ubicaciones consolidadas desde: Proyecto_Caso_Base\locations.csv
Calculando matriz de distancias...
Matriz de distancias calculada.

Matriz de Distancias (DataFrame):
           1          2          3          4          5          6   \
1    0.000000  17.182367  10.608396   6.370334  16.521196  10.565755   
2   17.182367   0.000000  10.270212  12.367369   0.803686   9.184090   
3   10.608396  10.270212   0.000000   4.239027   9.466781  10.798470   
4    6.370334  12.367369   4.239027   0.000000  11.603070   9.291240   
5   16.521196   0.803686   9.466781  11.603070   0.000000   8.884619   
6   10.565755   9.184090  10.798470   9.291240   8.884619   0.000000   
7    9.686597   7.501926   5.769840   5.454592   6.834748   5.089974   
8   15.435533  13.921205   6.024955   9.518335  13.194

In [7]:
from pyomo.environ import *
import pandas as pd
import os

N_ids = df_ubicaciones_para_matriz['LocationID'].tolist()
C_ids = df_clientes['LocationID'].tolist()
D_ids = df_depositos['LocationID'].tolist()
V_ids = df_vehiculos['VehicleID'].tolist()

if not D_ids:
    print("Error: No hay depósitos definidos.")
    exit()
deposito_principal_id = D_ids[0]

demanda_param = pd.Series(df_clientes['Demand'].values, index=df_clientes['LocationID']).to_dict()
capacidad_param = pd.Series(df_vehiculos['Capacity'].values, index=df_vehiculos['VehicleID']).to_dict()
autonomia_param = pd.Series(df_vehiculos['Range'].values, index=df_vehiculos['VehicleID']).to_dict()
dist_param = df_matriz_distancias.stack().to_dict()

modelo = ConcreteModel()

modelo.N_set = Set(initialize=N_ids)
modelo.C_set = Set(initialize=C_ids)
modelo.V_set = Set(initialize=V_ids)
modelo.D_set = Set(initialize=D_ids)

modelo.demanda = Param(modelo.C_set, initialize=demanda_param, default=0)
modelo.capacidad = Param(modelo.V_set, initialize=capacidad_param, default=0)
modelo.autonomia = Param(modelo.V_set, initialize=autonomia_param, default=0)
modelo.dist = Param(modelo.N_set, modelo.N_set, initialize=dist_param, default=999999)

modelo.x = Var(modelo.N_set, modelo.N_set, modelo.V_set, domain=Binary)
num_clientes_para_mtz = len(modelo.C_set)
modelo.u = Var(modelo.C_set, modelo.V_set, bounds=(1, num_clientes_para_mtz if num_clientes_para_mtz > 0 else 1), domain=Integers)

def objetivo_expr(m):
    return sum(m.dist[i,j] * m.x[i,j,k]
               for i in m.N_set for j in m.N_set for k in m.V_set if i != j)
modelo.obj = Objective(rule=objetivo_expr, sense=minimize)

modelo.rest_no_self_loops = ConstraintList()
for k in modelo.V_set:
    for i in modelo.N_set:
        modelo.rest_no_self_loops.add(modelo.x[i,i,k] == 0)

modelo.rest_visita_unica = ConstraintList()
for j_cliente in modelo.C_set:
    modelo.rest_visita_unica.add(
        sum(modelo.x[i,j_cliente,k] for i in modelo.N_set if i != j_cliente for k in modelo.V_set) == 1
    )

modelo.rest_salida_deposito = ConstraintList()
for k in modelo.V_set:
    modelo.rest_salida_deposito.add(
        sum(modelo.x[deposito_principal_id,j,k] for j in modelo.N_set if j != deposito_principal_id) <= 1
    )

modelo.rest_entrada_deposito = ConstraintList()
for k in modelo.V_set:
    modelo.rest_entrada_deposito.add(
        sum(modelo.x[i,deposito_principal_id,k] for i in modelo.N_set if i != deposito_principal_id) <= 1
    )

modelo.rest_salida_regreso_consistencia = ConstraintList()
for k in modelo.V_set:
    modelo.rest_salida_regreso_consistencia.add(
        sum(modelo.x[deposito_principal_id,j,k] for j in modelo.N_set if j != deposito_principal_id) ==
        sum(modelo.x[i,deposito_principal_id,k] for i in modelo.N_set if i != deposito_principal_id)
    )

modelo.rest_flujo = ConstraintList()
for k in modelo.V_set:
    for nodo_cliente in modelo.C_set:
        sum_entradas = sum(modelo.x[i,nodo_cliente,k] for i in modelo.N_set if i != nodo_cliente)
        sum_salidas = sum(modelo.x[nodo_cliente,j,k] for j in modelo.N_set if j != nodo_cliente)
        modelo.rest_flujo.add(sum_entradas == sum_salidas)

M_mtz = len(modelo.C_set) + 1 if len(modelo.C_set) > 0 else 2
modelo.rest_mtz = ConstraintList()
if len(modelo.C_set) > 1 :
    for k in modelo.V_set:
        for i_cliente in modelo.C_set:
            for j_cliente in modelo.C_set:
                if i_cliente != j_cliente:
                    modelo.rest_mtz.add(
                        modelo.u[i_cliente,k] - modelo.u[j_cliente,k] + M_mtz * modelo.x[i_cliente,j_cliente,k] <= M_mtz - 1
                    )

modelo.rest_capacidad = ConstraintList()
for k in modelo.V_set:
    modelo.rest_capacidad.add(
        sum(modelo.demanda[j_cliente] * sum(modelo.x[i,j_cliente,k] for i in modelo.N_set if i != j_cliente)
            for j_cliente in modelo.C_set) <= modelo.capacidad[k]
    )

modelo.rest_autonomia = ConstraintList()
for k in modelo.V_set:
    modelo.rest_autonomia.add(
        sum(modelo.dist[i,j] * modelo.x[i,j,k] for i in modelo.N_set for j in modelo.N_set if i != j) <= modelo.autonomia[k]
    )

solver = SolverFactory('glpk')
solver.options['tmlim'] = 300 # Límite de tiempo en segundos (300 seg = 5 minutos)
# solver.options['mipgap'] = 0.05 # Para algunos solvers, establece un gap de 5% (GLPK usa --mipgap en línea de comando)

resultados = solver.solve(modelo, tee=False)

if (resultados.solver.status == SolverStatus.ok) and \
   (resultados.solver.termination_condition == TerminationCondition.optimal or
    resultados.solver.termination_condition == TerminationCondition.feasible or # Si tmlim se alcanza y hay solución
    resultados.solver.termination_condition == TerminationCondition.maxTimeLimit): # Otra forma de indicar límite de tiempo
    print(f"Valor de la Función Objetivo (Distancia Total): {value(modelo.obj):.2f}")

    for k_vehiculo_id in modelo.V_set:
        vehiculo_usado_flag = any(value(modelo.x[deposito_principal_id, j_nodo_id, k_vehiculo_id]) > 0.5
                                  for j_nodo_id in modelo.N_set if j_nodo_id != deposito_principal_id)

        if not vehiculo_usado_flag:
            # No imprimir nada si el vehículo no se usó, para reducir la salida
            continue

        print(f"\nVehículo {k_vehiculo_id}:")
        ruta_actual_str = str(deposito_principal_id) 
        nodo_actual_id = deposito_principal_id
        distancia_vehiculo = 0
        demanda_atendida_vehiculo = 0
        nodos_visitados_en_ruta = {deposito_principal_id}

        for _ in range(len(modelo.N_set)):
            siguiente_nodo_encontrado = False
            for j_nodo_id in modelo.N_set:
                # Comprobar si el valor de x es suficientemente cercano a 1
                if j_nodo_id != nodo_actual_id and value(modelo.x[nodo_actual_id, j_nodo_id, k_vehiculo_id]) > 0.5:
                    if j_nodo_id in nodos_visitados_en_ruta and j_nodo_id != deposito_principal_id:
                        continue

                    ruta_actual_str += f" -> {str(j_nodo_id)}"
                    distancia_vehiculo += value(modelo.dist[nodo_actual_id, j_nodo_id])
                    if j_nodo_id in modelo.C_set: # Solo sumar demanda si es un cliente
                        demanda_atendida_vehiculo += value(modelo.demanda[j_nodo_id])
                    
                    nodo_actual_id = j_nodo_id
                    nodos_visitados_en_ruta.add(nodo_actual_id)
                    siguiente_nodo_encontrado = True
                    break

            if not siguiente_nodo_encontrado or nodo_actual_id == deposito_principal_id :
                # Asegurarse de que si la ruta no terminó explícitamente en el depósito pero debería, se imprima el regreso
                if nodo_actual_id != deposito_principal_id and any(value(modelo.x[nodo_actual_id, deposito_principal_id, k_vehiculo_id]) > 0.5 for _i in [0]):
                     ruta_actual_str += f" -> {deposito_principal_id}"
                     distancia_vehiculo += value(modelo.dist[nodo_actual_id, deposito_principal_id])
                break
        
        print(f"  Ruta: {ruta_actual_str}")
        print(f"  Distancia Recorrida: {distancia_vehiculo:.2f} (Autonomía: {value(modelo.autonomia[k_vehiculo_id])})")
        print(f"  Demanda Atendida: {demanda_atendida_vehiculo} (Capacidad: {value(modelo.capacidad[k_vehiculo_id])})")

elif resultados.solver.status == SolverStatus.warning and \
     resultados.solver.termination_condition == TerminationCondition.maxTimeLimit:
    print("\nLímite de Tiempo Alcanzado.")
    # Intentar imprimir el valor objetivo si existe una solución factible
    try:
        obj_val = value(modelo.obj)
        print(f"Valor de la Función Objetivo (Distancia Total) en el límite de tiempo: {obj_val:.2f}")
        # Aquí también podrías intentar imprimir la solución parcial como en el bloque 'if' anterior
    except Exception:
        print("No se pudo obtener el valor del objetivo (puede que no haya solución factible aún).")

else:
    print("\nNo se encontró una solución óptima o factible dentro de los criterios.")
    print(f"Estado del Solver: {resultados.solver.status}")
    print(f"Condición de Terminación: {resultados.solver.termination_condition}")

Valor de la Función Objetivo (Distancia Total): 178.81

Vehículo 1:
  Ruta: 1 -> 7 -> 2 -> 17 -> 11 -> 25 -> 18 -> 5 -> 16 -> 1
  Distancia Recorrida: 53.37 (Autonomía: 170)
  Demanda Atendida: 123 (Capacidad: 130)

Vehículo 2:
  Ruta: 1 -> 24 -> 4 -> 15 -> 13 -> 21 -> 1
  Distancia Recorrida: 19.50 (Autonomía: 200)
  Demanda Atendida: 69 (Capacidad: 140)

Vehículo 4:
  Ruta: 1 -> 14 -> 19 -> 12 -> 10 -> 23 -> 20 -> 1
  Distancia Recorrida: 57.95 (Autonomía: 90)
  Demanda Atendida: 99 (Capacidad: 100)

Vehículo 7:
  Ruta: 1 -> 9 -> 6 -> 3 -> 8 -> 22 -> 1
  Distancia Recorrida: 47.99 (Autonomía: 150)
  Demanda Atendida: 86 (Capacidad: 110)


# Casos

## Caso base

## Caso 2

### Carga de datos y preparacion para el caso 2

In [8]:
import pandas as pd
from pyomo.environ import *
import os
from geopy.distance import geodesic

directorio_datos_caso2 = 'caso2/'
ruta_clientes_caso2_csv = os.path.join(directorio_datos_caso2, 'clients.csv')
ruta_depositos_caso2_csv = os.path.join(directorio_datos_caso2, 'depots.csv')
ruta_vehiculos_caso2_csv = os.path.join(directorio_datos_caso2, 'vehicles.csv')

directorio_salida_caso2 = 'Proyecto_Caso2_Resultados'
if not os.path.exists(directorio_salida_caso2):
    os.makedirs(directorio_salida_caso2)

try:
    df_clientes_c2 = pd.read_csv(ruta_clientes_caso2_csv)
    df_depositos_c2 = pd.read_csv(ruta_depositos_caso2_csv)
    df_vehiculos_c2 = pd.read_csv(ruta_vehiculos_caso2_csv)
except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar un archivo de datos para el Caso 2: {e.filename}")
    raise

if 'Capacity' not in df_depositos_c2.columns:
    print("Error: El archivo depots.csv del Caso 2 debe tener una columna 'Capacity' para la capacidad de los depósitos.")


df_ubicaciones_consolidadas_c2 = pd.concat([
    df_depositos_c2[['LocationID', 'Longitude', 'Latitude']],
    df_clientes_c2[['LocationID', 'Longitude', 'Latitude']]
], ignore_index=True)

lista_ids_ubicaciones_c2 = df_ubicaciones_consolidadas_c2['LocationID'].tolist()
n_ubicaciones_c2 = len(lista_ids_ubicaciones_c2)
matriz_distancias_lista_c2 = [[0.0] * n_ubicaciones_c2 for _ in range(n_ubicaciones_c2)]

for i in range(n_ubicaciones_c2):
    fila_origen = df_ubicaciones_consolidadas_c2.iloc[i]
    coord_origen = (fila_origen['Latitude'], fila_origen['Longitude'])
    for j in range(n_ubicaciones_c2):
        if i == j:
            matriz_distancias_lista_c2[i][j] = 0.0
        else:
            fila_dest = df_ubicaciones_consolidadas_c2.iloc[j]
            coord_dest = (fila_dest['Latitude'], fila_dest['Longitude'])
            distancia_km = geodesic(coord_origen, coord_dest).kilometers
            matriz_distancias_lista_c2[i][j] = distancia_km

df_matriz_distancias_c2 = pd.DataFrame(
    matriz_distancias_lista_c2,
    index=lista_ids_ubicaciones_c2,
    columns=lista_ids_ubicaciones_c2
)

print("Datos del Caso 2 cargados y matriz de distancias calculada.")
print(f"Número de clientes: {len(df_clientes_c2)}")
print(f"Número de depósitos: {len(df_depositos_c2)}")
print(f"Número de vehículos: {len(df_vehiculos_c2)}")
if 'Capacity' in df_depositos_c2.columns:
    print("Capacidades de depósitos encontradas en el archivo.")
else:
    print("ADVERTENCIA: No se encontró la columna 'Capacity' en depots.csv. Se usará un valor por defecto alto.")

Datos del Caso 2 cargados y matriz de distancias calculada.
Número de clientes: 9
Número de depósitos: 12
Número de vehículos: 6
Capacidades de depósitos encontradas en el archivo.


### Modelo del caso 2 

In [ ]:
N_ids_c2 = df_ubicaciones_consolidadas_c2['LocationID'].tolist()
C_ids_c2 = df_clientes_c2['LocationID'].tolist()
D_ids_c2 = df_depositos_c2['LocationID'].tolist()
V_ids_c2 = df_vehiculos_c2['VehicleID'].tolist()

demanda_param_c2 = pd.Series(df_clientes_c2['Demand'].values, index=df_clientes_c2['LocationID']).to_dict()
capacidad_vehiculo_param_c2 = pd.Series(df_vehiculos_c2['Capacity'].values, index=df_vehiculos_c2['VehicleID']).to_dict()
autonomia_param_c2 = pd.Series(df_vehiculos_c2['Range'].values, index=df_vehiculos_c2['VehicleID']).to_dict()
dist_param_c2 = df_matriz_distancias_c2.stack().to_dict()

if 'Capacity' in df_depositos_c2.columns:
    capacidad_deposito_param_c2 = pd.Series(df_depositos_c2['Capacity'].values, index=df_depositos_c2['LocationID']).to_dict()
else:
    capacidad_deposito_param_c2 = {dep_id: float('inf') for dep_id in D_ids_c2}


modelo_c2 = ConcreteModel(name="MDVRP_Caso2")

modelo_c2.N_set = Set(initialize=N_ids_c2)
modelo_c2.C_set = Set(initialize=C_ids_c2)
modelo_c2.D_set = Set(initialize=D_ids_c2)
modelo_c2.V_set = Set(initialize=V_ids_c2)

modelo_c2.demanda = Param(modelo_c2.C_set, initialize=demanda_param_c2, default=0)
modelo_c2.capacidad_vehiculo = Param(modelo_c2.V_set, initialize=capacidad_vehiculo_param_c2, default=0)
modelo_c2.autonomia = Param(modelo_c2.V_set, initialize=autonomia_param_c2, default=0)
modelo_c2.dist = Param(modelo_c2.N_set, modelo_c2.N_set, initialize=dist_param_c2, default=999999)
modelo_c2.capacidad_deposito = Param(modelo_c2.D_set, initialize=capacidad_deposito_param_c2, default=float('inf'))

modelo_c2.x = Var(modelo_c2.N_set, modelo_c2.N_set, modelo_c2.V_set, domain=Binary)
num_clientes_para_mtz_c2 = len(modelo_c2.C_set)
modelo_c2.u = Var(modelo_c2.C_set, modelo_c2.V_set, 
                  bounds=(1, num_clientes_para_mtz_c2 if num_clientes_para_mtz_c2 > 0 else 1), 
                  domain=Integers)

modelo_c2.y_dk = Var(modelo_c2.D_set, modelo_c2.V_set, domain=Binary)


def objetivo_expr_c2(m):
    return sum(m.dist[i,j] * m.x[i,j,k]
               for i in m.N_set for j in m.N_set for k in m.V_set if i != j)
modelo_c2.obj = Objective(rule=objetivo_expr_c2, sense=minimize)


modelo_c2.rest_no_self_loops = ConstraintList()
for k in modelo_c2.V_set:
    for i in modelo_c2.N_set:
        modelo_c2.rest_no_self_loops.add(modelo_c2.x[i,i,k] == 0)

modelo_c2.rest_visita_unica = ConstraintList()
for j_cliente in modelo_c2.C_set:
    modelo_c2.rest_visita_unica.add(
        sum(modelo_c2.x[i,j_cliente,k] 
            for i in modelo_c2.N_set if i != j_cliente 
            for k in modelo_c2.V_set) == 1
    )

modelo_c2.rest_salida_un_deposito_por_vehiculo_y_dk = ConstraintList()
for k_veh in modelo_c2.V_set:
    modelo_c2.rest_salida_un_deposito_por_vehiculo_y_dk.add(sum(modelo_c2.y_dk[d_dep, k_veh] for d_dep in modelo_c2.D_set) <= 1)

modelo_c2.link_y_dk_con_salida_x = ConstraintList()
for d_dep in modelo_c2.D_set:
    for k_veh in modelo_c2.V_set:
        modelo_c2.link_y_dk_con_salida_x.add(
            sum(modelo_c2.x[d_dep, j_cli, k_veh] for j_cli in modelo_c2.C_set) <= modelo_c2.y_dk[d_dep, k_veh] * len(modelo_c2.C_set)
        )
        modelo_c2.link_y_dk_con_salida_x.add(
             modelo_c2.y_dk[d_dep, k_veh] <= sum(modelo_c2.x[d_dep, j_cli, k_veh] for j_cli in modelo_c2.C_set) 
        )

modelo_c2.rest_regreso_mismo_deposito = ConstraintList()
for k_veh in modelo_c2.V_set:
    for d_origen in modelo_c2.D_set:
        sum_salidas_de_d_a_clientes = sum(modelo_c2.x[d_origen, j_cli, k_veh] for j_cli in modelo_c2.C_set)
        sum_entradas_a_d_desde_clientes = sum(modelo_c2.x[i_cli, d_origen, k_veh] for i_cli in modelo_c2.C_set)
        modelo_c2.rest_regreso_mismo_deposito.add(sum_salidas_de_d_a_clientes == sum_entradas_a_d_desde_clientes)
        modelo_c2.rest_regreso_mismo_deposito.add(modelo_c2.y_dk[d_origen, k_veh] == sum_salidas_de_d_a_clientes)


modelo_c2.rest_no_arcos_entre_depositos = ConstraintList()
for k in modelo_c2.V_set:
    for d1 in modelo_c2.D_set:
        for d2 in modelo_c2.D_set:
            if d1 != d2:
                modelo_c2.rest_no_arcos_entre_depositos.add(modelo_c2.x[d1,d2,k] == 0)
                modelo_c2.rest_no_arcos_entre_depositos.add(modelo_c2.x[d2,d1,k] == 0)


modelo_c2.rest_flujo_clientes = ConstraintList()
for k in modelo_c2.V_set:
    for nodo_cliente in modelo_c2.C_set:
        sum_entradas = sum(modelo_c2.x[i,nodo_cliente,k] for i in modelo_c2.N_set if i != nodo_cliente)
        sum_salidas = sum(modelo_c2.x[nodo_cliente,j,k] for j in modelo_c2.N_set if j != nodo_cliente)
        modelo_c2.rest_flujo_clientes.add(sum_entradas == sum_salidas)


M_mtz_c2 = len(modelo_c2.C_set) + 1 if len(modelo_c2.C_set) > 0 else 2
modelo_c2.rest_mtz = ConstraintList()
if len(modelo_c2.C_set) > 1 :
    for k in modelo_c2.V_set:
        for i_cliente in modelo_c2.C_set:
            for j_cliente in modelo_c2.C_set:
                if i_cliente != j_cliente:
                    modelo_c2.rest_mtz.add(
                        modelo_c2.u[i_cliente,k] - modelo_c2.u[j_cliente,k] + M_mtz_c2 * modelo_c2.x[i_cliente,j_cliente,k] <= M_mtz_c2 - 1
                    )

modelo_c2.rest_capacidad_vehiculo = ConstraintList()
for k in modelo_c2.V_set:
    modelo_c2.rest_capacidad_vehiculo.add(
        sum(modelo_c2.demanda[j_cliente] * sum(modelo_c2.x[i,j_cliente,k] 
                                               for i in modelo_c2.N_set if i != j_cliente)
            for j_cliente in modelo_c2.C_set) <= modelo_c2.capacidad_vehiculo[k]
    )

modelo_c2.rest_autonomia = ConstraintList()
for k in modelo_c2.V_set:
    modelo_c2.rest_autonomia.add(
        sum(modelo_c2.dist[i,j] * modelo_c2.x[i,j,k] 
            for i in modelo_c2.N_set 
            for j in modelo_c2.N_set if i != j) <= modelo_c2.autonomia[k]
    )

modelo_c2.carga_vehiculo_k_expr = Var(modelo_c2.V_set, domain=NonNegativeReals)
modelo_c2.def_carga_vehiculo_k_expr = ConstraintList()
for k_veh in modelo_c2.V_set:
    modelo_c2.def_carga_vehiculo_k_expr.add(
        modelo_c2.carga_vehiculo_k_expr[k_veh] == sum(
            modelo_c2.demanda[j_cli] * sum(modelo_c2.x[i_nod, j_cli, k_veh] for i_nod in modelo_c2.N_set if i_nod != j_cli)
            for j_cli in modelo_c2.C_set
        )
    )

modelo_c2.z_dk_carga_asignada = Var(modelo_c2.D_set, modelo_c2.V_set, domain=NonNegativeReals)
modelo_c2.def_z_dk_linealizada = ConstraintList()
M_cap_grande = sum(d for d in demanda_param_c2.values()) if demanda_param_c2 else 10000 

for d_dep in modelo_c2.D_set:
    for k_veh in modelo_c2.V_set:
        modelo_c2.def_z_dk_linealizada.add(modelo_c2.z_dk_carga_asignada[d_dep, k_veh] <= modelo_c2.carga_vehiculo_k_expr[k_veh])
        modelo_c2.def_z_dk_linealizada.add(modelo_c2.z_dk_carga_asignada[d_dep, k_veh] <= M_cap_grande * modelo_c2.y_dk[d_dep, k_veh])
        modelo_c2.def_z_dk_linealizada.add(modelo_c2.z_dk_carga_asignada[d_dep, k_veh] >= modelo_c2.carga_vehiculo_k_expr[k_veh] - M_cap_grande * (1 - modelo_c2.y_dk[d_dep, k_veh]))
        modelo_c2.def_z_dk_linealizada.add(modelo_c2.z_dk_carga_asignada[d_dep, k_veh] >= 0)


modelo_c2.rest_capacidad_deposito_linealizada = ConstraintList()
for d_dep in modelo_c2.D_set:
     modelo_c2.rest_capacidad_deposito_linealizada.add(
         sum(modelo_c2.z_dk_carga_asignada[d_dep, k_veh] for k_veh in modelo_c2.V_set) <= modelo_c2.capacidad_deposito[d_dep]
     )


solver_c2 = SolverFactory('glpk')
solver_c2.options['tmlim'] = 300 
print("\nIniciando resolución del modelo MDVRP (Caso 2)...")
resultados_c2 = solver_c2.solve(modelo_c2, tee=False)


if (resultados_c2.solver.status == SolverStatus.ok or resultados_c2.solver.status == SolverStatus.warning) and \
   (resultados_c2.solver.termination_condition == TerminationCondition.optimal or
    resultados_c2.solver.termination_condition == TerminationCondition.feasible or
    resultados_c2.solver.termination_condition == TerminationCondition.maxTimeLimit):
    
    print(f"\n--- Solución MDVRP (Caso 2) Encontrada (o parcialmente encontrada) ---")
    try:
        obj_val_c2 = value(modelo_c2.obj)
        print(f"Valor de la Función Objetivo (Distancia Total): {obj_val_c2:.2f}")
    except Exception as e:
        print(f"No se pudo obtener el valor del objetivo. Puede que no haya una solución factible completa. Error: {e}")
                
    rutas_por_deposito = {dep_id: [] for dep_id in modelo_c2.D_set}
    
    for k_vehiculo_id in modelo_c2.V_set:
        origen_dep = None
        for d_dep_id in modelo_c2.D_set:
            variable_y_dk = modelo_c2.y_dk[d_dep_id, k_vehiculo_id]
            # --- MODIFICACIÓN AQUÍ ---
            if variable_y_dk.value is not None and variable_y_dk.value > 0.5:
            # --- FIN DE MODIFICACIÓN ---
                origen_dep = d_dep_id
                break 
        
        if not origen_dep:
            continue

        ruta_actual_str_list = [str(origen_dep)]
        nodo_actual_id = origen_dep
        distancia_vehiculo = 0
        demanda_atendida_vehiculo = 0
        nodos_visitados_en_ruta = {origen_dep}

        for _ in range(len(modelo_c2.N_set)): 
            siguiente_nodo_encontrado = False
            for j_nodo_id in modelo_c2.N_set:
                # --- Comprobación adicional para x ---
                variable_x = modelo_c2.x[nodo_actual_id, j_nodo_id, k_vehiculo_id]
                if j_nodo_id != nodo_actual_id and variable_x.value is not None and variable_x.value > 0.5:
                # --- Fin comprobación para x ---
                    if j_nodo_id in nodos_visitados_en_ruta and j_nodo_id != origen_dep :
                        continue

                    ruta_actual_str_list.append(str(j_nodo_id))
                    
                    # Comprobación para dist y demanda antes de value()
                    dist_val = modelo_c2.dist[nodo_actual_id, j_nodo_id]
                    distancia_vehiculo += dist_val # Los parámetros deben tener valor o default

                    if j_nodo_id in modelo_c2.C_set:
                        demanda_val = modelo_c2.demanda[j_nodo_id]
                        demanda_atendida_vehiculo += demanda_val
                    
                    nodo_actual_id = j_nodo_id
                    nodos_visitados_en_ruta.add(nodo_actual_id)
                    siguiente_nodo_encontrado = True
                    break  
            
            if not siguiente_nodo_encontrado or nodo_actual_id == origen_dep:
                # Asegurar que la ruta impresa termine en el depósito de origen si no lo hizo explícitamente
                if ruta_actual_str_list[-1] != str(origen_dep):
                    var_x_regreso = modelo_c2.x[nodo_actual_id, origen_dep, k_vehiculo_id]
                    if var_x_regreso.value is not None and var_x_regreso.value > 0.5:
                         ruta_actual_str_list.append(str(origen_dep))
                         dist_val_regreso = modelo_c2.dist[nodo_actual_id, origen_dep]
                         distancia_vehiculo += dist_val_regreso
                break
        
        if len(ruta_actual_str_list) > 1 :
            # Los parámetros como autonomia y capacidad_vehiculo deberían tener valor
            autonomia_val = value(modelo_c2.autonomia[k_vehiculo_id])
            cap_veh_val = value(modelo_c2.capacidad_vehiculo[k_vehiculo_id])

            rutas_por_deposito[origen_dep].append({
                 "Vehiculo": k_vehiculo_id, 
                 "Ruta": " -> ".join(ruta_actual_str_list), 
                 "DemandaVehiculo": demanda_atendida_vehiculo,
                 "DistanciaVehiculo": round(distancia_vehiculo,2),
                 "CapacidadVehiculo": cap_veh_val,
                 "AutonomiaVehiculo": autonomia_val
            })

    for dep_id, infos_rutas in rutas_por_deposito.items():
        if not infos_rutas: continue
    
        cap_dep_val = value(modelo_c2.capacidad_deposito[dep_id]) 
        print(f"\n--- Depósito: {dep_id} (Capacidad Dep.: {cap_dep_val if cap_dep_val != float('inf') else 'Infinita'}) ---")
        demanda_total_deposito = 0
        for info_r in infos_rutas:
            print(f"  Vehículo {info_r['Vehiculo']}: {info_r['Ruta']}")
            print(f"    Demanda: {info_r['DemandaVehiculo']} (Cap. Veh: {info_r['CapacidadVehiculo']}), Distancia: {info_r['DistanciaVehiculo']} (Aut: {info_r['AutonomiaVehiculo']})")
            demanda_total_deposito += info_r['DemandaVehiculo']
        print(f"  Demanda Total Servida desde Depósito {dep_id}: {demanda_total_deposito}")
        if cap_dep_val != float('inf') and demanda_total_deposito > cap_dep_val:
                 print(f"  ALERTA: Demanda del depósito ({demanda_total_deposito}) excede su capacidad ({cap_dep_val})!")


elif resultados_c2.solver.status == SolverStatus.warning and \
     resultados_c2.solver.termination_condition == TerminationCondition.maxTimeLimit:
    print("\n--- Límite de Tiempo Alcanzado para MDVRP (Caso 2) ---")
    try:
        obj_val_c2 = value(modelo_c2.obj)
        print(f"Valor de la Función Objetivo (Distancia Total) en el límite de tiempo: {obj_val_c2:.2f}")
    except Exception:
        print("No se pudo obtener el valor del objetivo.")
else:
    print("\n--- No se encontró una solución óptima o factible para MDVRP (Caso 2) ---")
    print(f"Estado del Solver: {resultados_c2.solver.status}")
    print(f"Condición de Terminación: {resultados_c2.solver.termination_condition}")


Iniciando resolución del modelo MDVRP (Caso 2)...

--- Solución MDVRP (Caso 2) Encontrada ---
ERROR: evaluating object as numeric value: x[1,2,1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[1,2,1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[1,2,1]
No se pudo obtener el valor del objetivo. Error: No value for uninitialized NumericValue object x[1,2,1]
ERROR: evaluating object as numeric value: y_dk[1,1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object y_dk[1,1]


ValueError: No value for uninitialized NumericValue object y_dk[1,1]

## Caso b

# Análisis de resultados y visualizaciones

## Visualizacion caso base

In [ ]:
import pandas as pd
from pyomo.environ import value
import os

VELOCIDAD_PROMEDIO_KMH = 30
COSTO_COMBUSTIBLE_POR_KM = 1500

HEADERS_VERIFICACION = [
    "VehicleId", "DepotId", "InitialLoad", "RouteSequence", "ClientsServed",
    "DemandsSatisfied", "TotalDistance", "TotalTime", "FuelCost"
]

datos_verificacion_caso_base = []

directorio_salida_caso_base = 'Proyecto_Caso_Base_Resultados'
if not os.path.exists(directorio_salida_caso_base):
    os.makedirs(directorio_salida_caso_base)

if (resultados.solver.status == SolverStatus.ok) and \
   (resultados.solver.termination_condition == TerminationCondition.optimal or
    resultados.solver.termination_condition == TerminationCondition.feasible or
    resultados.solver.termination_condition == TerminationCondition.maxTimeLimit):

    print("Generando archivo de verificación para Caso Base...")

    for k_vehiculo_id in modelo.V_set:
        vehiculo_usado_flag = any(value(modelo.x[deposito_principal_id, j_nodo_id, k_vehiculo_id]) > 0.5
                                  for j_nodo_id in modelo.N_set if j_nodo_id != deposito_principal_id)

        if not vehiculo_usado_flag:
            continue
        
        origen_dep_id = deposito_principal_id 

        ruta_nodos_ids = [str(origen_dep_id)]
        clientes_en_ruta_ids = []
        demandas_satisfechas_lista = []
        
        nodo_actual_id = origen_dep_id
        distancia_total_ruta = 0.0
        tiempo_total_ruta_minutos = 0.0

        for _ in range(len(modelo.N_set) + 1):
            siguiente_nodo_encontrado = False
            for j_nodo_id in modelo.N_set:
                if j_nodo_id != nodo_actual_id and value(modelo.x[nodo_actual_id, j_nodo_id, k_vehiculo_id]) > 0.5:
                    dist_segmento = value(modelo.dist[nodo_actual_id, j_nodo_id])
                    distancia_total_ruta += dist_segmento

                    if VELOCIDAD_PROMEDIO_KMH > 0:
                        tiempo_segmento_horas = dist_segmento / VELOCIDAD_PROMEDIO_KMH
                        tiempo_segmento_minutos = tiempo_segmento_horas * 60
                    else:
                        tiempo_segmento_minutos = 0
                    tiempo_total_ruta_minutos += tiempo_segmento_minutos

                    ruta_nodos_ids.append(str(j_nodo_id))
                    
                    if j_nodo_id in modelo.C_set:
                        clientes_en_ruta_ids.append(j_nodo_id)
                        demandas_satisfechas_lista.append(str(int(value(modelo.demanda[j_nodo_id]))))

                    nodo_actual_id = j_nodo_id
                    siguiente_nodo_encontrado = True
                    break
            
            if not siguiente_nodo_encontrado:
                break 
            
            if nodo_actual_id == origen_dep_id and len(ruta_nodos_ids) > 1: 
                break
        
        if len(ruta_nodos_ids) == 1 and ruta_nodos_ids[0] == str(origen_dep_id):
             continue

        vehicle_id_str = str(k_vehiculo_id)
        depot_id_str = str(origen_dep_id)
        
        initial_load_val = sum(value(modelo.demanda[cliente_id]) for cliente_id in clientes_en_ruta_ids)
        
        route_sequence_str = " - ".join(ruta_nodos_ids)
        clients_served_count = len(clientes_en_ruta_ids)
        demands_satisfied_str = "-".join(demandas_satisfechas_lista) if demandas_satisfechas_lista else ""
        
        total_distance_str = f"{distancia_total_ruta:.1f}"
        total_time_str = f"{tiempo_total_ruta_minutos:.1f}"
        
        fuel_cost_val = distancia_total_ruta * COSTO_COMBUSTIBLE_POR_KM
        fuel_cost_str = str(int(round(fuel_cost_val)))

        datos_verificacion_caso_base.append({
            "VehicleId": vehicle_id_str,
            "DepotId": depot_id_str,
            "InitialLoad": int(round(initial_load_val)),
            "RouteSequence": route_sequence_str,
            "ClientsServed": clients_served_count,
            "DemandsSatisfied": demands_satisfied_str,
            "TotalDistance": total_distance_str,
            "TotalTime": total_time_str,
            "FuelCost": fuel_cost_str
        })

    if datos_verificacion_caso_base:
        df_verificacion_base = pd.DataFrame(datos_verificacion_caso_base, columns=HEADERS_VERIFICACION)
        ruta_archivo_verificacion_base = os.path.join(directorio_salida_caso_base, "verificacion_caso_base.csv") # o verificacion_caso1.csv
        df_verificacion_base.to_csv(ruta_archivo_verificacion_base, index=False, sep=',')
        print(f"Archivo de verificación '{os.path.basename(ruta_archivo_verificacion_base)}' generado exitosamente en '{directorio_salida_caso_base}'.")
    else:
        print("No se generaron rutas válidas para incluir en el archivo de verificación del Caso Base.")

else:
    print("No se encontró solución para el Caso Base, no se generará archivo de verificación.")

## Visualizacion caso 2

In [ ]:
import pandas as pd
from pyomo.environ import value
import os

VELOCIDAD_PROMEDIO_KMH = 30
COSTO_COMBUSTIBLE_POR_KM = 1500

HEADERS_VERIFICACION = [
    "VehicleId", "DepotId", "InitialLoad", "RouteSequence", "ClientsServed",
    "DemandsSatisfied", "TotalDistance", "TotalTime", "FuelCost"
]

datos_verificacion_caso2 = []

if (resultados_c2.solver.status == SolverStatus.ok) and \
   (resultados_c2.solver.termination_condition == TerminationCondition.optimal or
    resultados_c2.solver.termination_condition == TerminationCondition.feasible or
    resultados_c2.solver.termination_condition == TerminationCondition.maxTimeLimit):

    print("Generando archivo de verificación para Caso 2...")

    for k_vehiculo_id in modelo_c2.V_set:
        origen_dep_id = None
        for d_dep_id_loop in modelo_c2.D_set:
            if value(modelo_c2.y_dk[d_dep_id_loop, k_vehiculo_id]) > 0.5:
                origen_dep_id = d_dep_id_loop
                break
        
        if not origen_dep_id:
            continue

        ruta_nodos_ids = [str(origen_dep_id)] 
        clientes_en_ruta_ids = []
        demandas_satisfechas_lista = []
        
        nodo_actual_id = origen_dep_id
        distancia_total_ruta = 0.0
        tiempo_total_ruta_minutos = 0.0

        for _ in range(len(modelo_c2.N_set) + 1):
            siguiente_nodo_encontrado = False
            for j_nodo_id in modelo_c2.N_set:
                if j_nodo_id != nodo_actual_id and value(modelo_c2.x[nodo_actual_id, j_nodo_id, k_vehiculo_id]) > 0.5:
                    dist_segmento = value(modelo_c2.dist[nodo_actual_id, j_nodo_id])
                    distancia_total_ruta += dist_segmento

                    if VELOCIDAD_PROMEDIO_KMH > 0:
                        tiempo_segmento_horas = dist_segmento / VELOCIDAD_PROMEDIO_KMH
                        tiempo_segmento_minutos = tiempo_segmento_horas * 60
                    else:
                        tiempo_segmento_minutos = 0
                    tiempo_total_ruta_minutos += tiempo_segmento_minutos

                    ruta_nodos_ids.append(str(j_nodo_id))
                    
                    if j_nodo_id in modelo_c2.C_set:
                        clientes_en_ruta_ids.append(j_nodo_id)
                        demandas_satisfechas_lista.append(str(int(value(modelo_c2.demanda[j_nodo_id]))))

                    nodo_actual_id = j_nodo_id
                    siguiente_nodo_encontrado = True
                    break
            
            if not siguiente_nodo_encontrado:
                break 
            
            if nodo_actual_id == origen_dep_id and len(ruta_nodos_ids) > 1: 
                break
        
        if len(ruta_nodos_ids) == 1 and ruta_nodos_ids[0] == str(origen_dep_id):
             continue

        vehicle_id_str = str(k_vehiculo_id)
        depot_id_str = str(origen_dep_id)
        
        initial_load_val = sum(value(modelo_c2.demanda[cliente_id]) for cliente_id in clientes_en_ruta_ids)
        
        route_sequence_str = " - ".join(ruta_nodos_ids)
        clients_served_count = len(clientes_en_ruta_ids)
        demands_satisfied_str = "-".join(demandas_satisfechas_lista) if demandas_satisfechas_lista else ""
        
        total_distance_str = f"{distancia_total_ruta:.1f}"
        total_time_str = f"{tiempo_total_ruta_minutos:.1f}"
        
        fuel_cost_val = distancia_total_ruta * COSTO_COMBUSTIBLE_POR_KM
        fuel_cost_str = str(int(round(fuel_cost_val)))

        datos_verificacion_caso2.append({
            "VehicleId": vehicle_id_str,
            "DepotId": depot_id_str,
            "InitialLoad": int(round(initial_load_val)),
            "RouteSequence": route_sequence_str,
            "ClientsServed": clients_served_count,
            "DemandsSatisfied": demands_satisfied_str,
            "TotalDistance": total_distance_str,
            "TotalTime": total_time_str,
            "FuelCost": fuel_cost_str
        })

    if datos_verificacion_caso2:
        df_verificacion = pd.DataFrame(datos_verificacion_caso2, columns=HEADERS_VERIFICACION)
        # Asumimos que 'directorio_salida_caso2' fue definido en la celda anterior
        # Si no, definirlo aquí:
        # directorio_salida_caso2 = 'Proyecto_Caso2_Resultados' 
        # if not os.path.exists(directorio_salida_caso2):
        # os.makedirs(directorio_salida_caso2)
        ruta_archivo_verificacion_c2 = os.path.join(directorio_salida_caso2, "verificacion_caso2.csv")
        df_verificacion.to_csv(ruta_archivo_verificacion_c2, index=False, sep=',')
        print(f"Archivo de verificación 'verificacion_caso2.csv' generado exitosamente en '{directorio_salida_caso2}'.")
    else:
        print("No se generaron rutas válidas para incluir en el archivo de verificación del Caso 2.")

else:
    print("No se encontró solución para el Caso 2, no se generará archivo de verificación.")